In [2]:
!pip install orjson

  Using cached orjson-3.9.15-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (49 kB)
Using cached orjson-3.9.15-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (138 kB)

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [3]:
import pandas as pd
import numpy as np
import json
import os
import orjson
import glob
import sys
import time
from collections import defaultdict
from datetime import datetime,timedelta
import math
from collections import Counter
import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_rows',None)
pd.set_option('display.max_columns',None)
pd.set_option('display.max_colwidth', None)

In [ ]:
### Reading Data
df= pd.read_csv('s3://scien-ds-sm-root-backup/home/sushil/LN_Unified_V2/Client_Data_Auto/210324_LN_V2_Auto_Dataset_With_Performance_Race_Age_and_Gender.csv')
df.shape

In [ ]:
df[['approve_flag', 'funded_flag', 'bad_flag']].sum()

In [ ]:
df['client'].value_counts(dropna= False)

In [ ]:
df= df[df['client']!= 'AAC'].reset_index(drop= True)
df.shape

In [ ]:
df= df[df['auto_score']>500].reset_index(drop= True)
df.shape

In [ ]:
df['app_date']= df['app_date'].apply(lambda x: pd.to_datetime(x[:10]) if pd.isna(x)== False else x)
df['app_date'].describe()

In [28]:
def get_age(app_date, dob):
#     app_date= app_date.copy()
#     dob= dob.copy()
    
    if (pd.isna(app_date)== False) and (pd.isna(dob)== False):
        
        app_date= pd.to_datetime(app_date)
        dob= pd.to_datetime(dob)
        
        if dob >= app_date:
            dob= dob.replace(year= dob.year - 100)
        else:
            dob= dob

        app_year, app_month, app_day= app_date.year, app_date.month, app_date.day
        dob_year, dob_month, dob_day= dob.year, dob.month, dob.day
        
        age= (app_year - dob_year) + ((app_month - dob_month)/12) + ((app_day - dob_day)/365.5)
        
        return age
    
    else:
        return np.NaN

In [29]:
df[df['applicant_age']<0].shape

(70481, 342)

In [30]:
df[df['applicant_age']<0][['client', 'applicant_age', 'applicant_dob', 'app_date']].sample(10)

,client,applicant_age,applicant_dob,app_date
732066,Tower,-51.609211,11/10/69,2018-03-31
698631,Tower,-54.105221,09/22/71,2017-08-14
761345,Tower,-53.961696,11/10/72,2018-11-24
801776,Tower,-43.789558,06/09/64,2020-08-25
823648,Tower,-31.659941,01/29/53,2021-06-01
783887,Tower,-52.980848,09/20/72,2019-09-27
716347,Tower,-45.777360,09/25/63,2017-12-15
801019,Tower,-39.866165,06/24/60,2020-08-12
665431,Tower,-33.857957,11/23/50,2017-01-14
773083,Tower,-37.295030,08/13/56,2019-04-27


In [31]:
df['applicant_age']= df.apply(lambda x: get_age(x['app_date'], x['applicant_dob']) if x['applicant_age']< 0 else x['applicant_age'], axis= 1)
df['applicant_age'].describe()

count    821726.000000
mean         42.632391
std          14.738996
min           0.000000
25%          30.166667
50%          41.937073
75%          52.639307
max         228.099749
Name: applicant_age, dtype: float64

In [32]:
df.loc[732066][['client', 'applicant_age', 'applicant_dob', 'app_date']]

client                         Tower
applicant_age              48.390789
applicant_dob               11/10/69
app_date         2018-03-31 00:00:00
Name: 732066, dtype: object

In [33]:
# df.to_csv('/root/Unified_LN_V2/LN_Auto_V2/Data/1_LN_Auto_All_Updated_Data_220324.csv', index= False)

In [3]:
df= pd.read_csv('/root/Unified_LN_V2/LN_Auto_V2/Data/1_LN_Auto_All_Updated_Data_220324.csv')
df.shape

(824212, 342)

In [4]:
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit, StratifiedGroupKFold, StratifiedKFold

In [5]:
# sgkf= StratifiedGroupKFold(n_splits= 5, shuffle= True, random_state= 127)

In [18]:
T1,T2= train_test_split(df, test_size= 0.15, random_state= 2022)
T1.shape, T2.shape

((700580, 342), (123632, 342))

In [19]:
T1[['approve_flag', 'funded_flag', 'bad_flag']].sum(), T2[['approve_flag', 'funded_flag', 'bad_flag']].sum()

(approve_flag    412873.0
 funded_flag     209793.0
 bad_flag          3186.0
 dtype: float64,
 approve_flag    72832.0
 funded_flag     37121.0
 bad_flag          545.0
 dtype: float64)

In [20]:
T1['bad_flag'].sum()/T1['funded_flag'].sum(), T2['bad_flag'].sum()/T2['funded_flag'].sum()

(0.015186398020906322, 0.01468171654858436)

In [21]:
T1['approve_flag'].mean(), T2['approve_flag'].mean()

(0.5893302692055155, 0.5891031448168759)

In [22]:
T1.groupby('client').agg(
    ttd= ('unique_id', 'count'),
    approved= ('approve_flag', 'sum'),
    funded= ('funded_flag', 'sum'),
    bad= ('bad_flag', 'sum'),
    avg_fico= ('fico_pb', 'mean'),
    avg_auto_score= ('auto_score', 'mean'),
    min_app_date=  ('app_date', 'min'),
    max_app_date= ('app_date', 'max')
)

,ttd,approved,funded,bad,avg_fico,avg_auto_score,min_app_date,max_app_date
client,,,,,,,,
Arkansas,147800,87742.0,39869.0,241.0,680.155968,719.383917,2020-01-01,2022-09-29
Bethpage,32968,22737.0,11837.0,84.0,717.302632,719.934725,2017-01-01,2019-06-30
CPM,15365,11268.0,9645.0,139.0,695.021998,730.526782,2016-12-05,2022-09-29
GESA,80506,48766.0,28069.0,586.0,654.664366,701.111892,2018-01-01,2019-09-14
Guardian,82074,43060.0,27406.0,391.0,611.390135,707.273095,2019-11-05,2022-09-29
HFCU,14129,7406.0,2895.0,55.0,644.605763,719.001769,2018-11-27,2020-05-16
Hawaii,5973,2117.0,2077.0,4.0,625.453876,705.021430,2019-01-02,2022-09-29
NCU,2470,1631.0,873.0,12.0,588.333198,703.585020,2019-01-02,2019-11-29
Numerica,128156,60009.0,25791.0,419.0,646.034852,710.943296,2017-01-20,2020-01-31


In [23]:
T2.groupby('client').agg(
    ttd= ('unique_id', 'count'),
    approved= ('approve_flag', 'sum'),
    funded= ('funded_flag', 'sum'),
    bad= ('bad_flag', 'sum'),
    avg_fico= ('fico_pb', 'mean'),
    avg_auto_score= ('auto_score', 'mean'),
    min_app_date=  ('app_date', 'min'),
    max_app_date= ('app_date', 'max')
)

,ttd,approved,funded,bad,avg_fico,avg_auto_score,min_app_date,max_app_date
client,,,,,,,,
Arkansas,26286,15655.0,7081.0,50.0,681.427680,719.601080,2020-01-01,2022-09-29
Bethpage,5815,4027.0,2128.0,12.0,718.054710,720.316251,2017-01-02,2019-06-30
CPM,2640,1949.0,1685.0,21.0,697.906818,730.249621,2016-12-20,2022-09-29
GESA,14159,8602.0,4937.0,103.0,656.734236,701.559503,2018-01-01,2019-09-14
Guardian,14553,7581.0,4904.0,48.0,608.410010,707.152065,2019-11-05,2022-09-29
HFCU,2467,1287.0,515.0,9.0,641.759623,717.968788,2018-11-28,2020-05-16
Hawaii,1045,377.0,366.0,2.0,617.253589,706.092823,2019-01-02,2022-09-28
NCU,438,289.0,143.0,4.0,587.394977,702.121005,2019-01-02,2019-11-29
Numerica,22583,10562.0,4515.0,92.0,646.125305,710.990524,2017-01-20,2020-01-31


In [24]:
# T2.to_csv('/root/Unified_LN_V2/LN_Auto_V2/Data/1_LN_Auto_V2_OOT_Data_220324.csv', index= False)
# T1.to_csv('/root/Unified_LN_V2/LN_Auto_V2/Data/1_LN_Auto_V2_DEV_Data_220324.csv', index= False)

In [25]:
del df, T2

### Now Splitting the data into Train and Test

In [26]:
T1.shape

(700580, 342)

In [27]:
T1['Borrower_Race_Prim_BISG'].value_counts(dropna= False)

Borrower_Race_Prim_BISG
white       386048
missing     188851
black        60934
hispanic     55029
api           8771
aian           816
multiple       128
others           3
Name: count, dtype: int64

In [28]:
T1['borrower_gender'].value_counts(dropna= False)

borrower_gender
M    383258
F    265313
I     52009
Name: count, dtype: int64

In [29]:
T1['Race']= T1['Borrower_Race_Prim_BISG']
T1['Gender']= T1['borrower_gender']
T1['age_cuts']= np.where(T1['applicant_age'].isnull(), 'missing',
                         np.where(T1['applicant_age']< 62, 'Age below 62',
                                  np.where(T1['applicant_age']>= 62, 'Age 62 and over', 'missing')))

T1['age_cuts'].value_counts(dropna= False)

age_cuts
Age below 62       620625
Age 62 and over     77829
missing              2126
Name: count, dtype: int64

In [30]:
T1.groupby('age_cuts')['applicant_age'].agg(['min', 'max'])

,min,max
age_cuts,,
Age 62 and over,62.0,228.099749
Age below 62,0.0,61.997264
missing,NaN,NaN


In [59]:
T3,T4= train_test_split(T1, test_size= 0.3, random_state= 139)
T3.shape, T4.shape

((490406, 345), (210174, 345))

In [60]:
T3[['approve_flag', 'funded_flag', 'bad_flag']].sum(), T4[['approve_flag', 'funded_flag', 'bad_flag']].sum()

(approve_flag    288932.0
 funded_flag     146675.0
 bad_flag          2221.0
 dtype: float64,
 approve_flag    123941.0
 funded_flag      63118.0
 bad_flag           965.0
 dtype: float64)

In [61]:
T3['approve_flag'].mean(), T4['approve_flag'].mean()

(0.589168974278455, 0.5897066240353231)

In [62]:
# T3.to_csv('/root/Unified_LN_V2/LN_Auto_V2/Data/1_LN_Auto_V2_Train_RI_Data_220324.csv', index= False)
# T4.to_csv('/root/Unified_LN_V2/LN_Auto_V2/Data/1_LN_Auto_V2_Test_RI_Data_220324.csv', index= False)

### Next Part Funded

In [4]:
Train = pd.read_csv('/root/Unified_LN_V2/LN_Auto_V2/Data/1_LN_Auto_V2_Train_RI_Data_220324.csv')
Test = pd.read_csv('/root/Unified_LN_V2/LN_Auto_V2/Data/1_LN_Auto_V2_Test_RI_Data_220324.csv')
Oot= pd.read_csv('/root/Unified_LN_V2/LN_Auto_V2/Data/1_LN_Auto_V2_OOT_Data_220324.csv')

Train.shape, Test.shape, Oot.shape

((490406, 345), (210174, 345), (123632, 342))

In [5]:
Oot['Race']= Oot['Borrower_Race_Prim_BISG']
Oot['Gender']= Oot['borrower_gender']
Oot['age_cuts']= np.where(Oot['applicant_age'].isnull(), 'missing',
                         np.where(Oot['applicant_age']< 62, 'Age below 62',
                                  np.where(Oot['applicant_age']>= 62, 'Age 62 and over', 'missing')))

Oot['age_cuts'].value_counts(dropna= False)

age_cuts
Age below 62       109758
Age 62 and over     13514
missing               360
Name: count, dtype: int64

In [6]:
TTD= pd.concat([Train, Test]).reset_index(drop= True)
TTD.shape

(700580, 345)

In [7]:
funded_dev= TTD[TTD['funded_flag']== 1].reset_index(drop= True)
funded_dev.shape

(209793, 345)

In [8]:
scored= pd.read_csv('/root/Unified_LN_V2/LN_Auto_V2/Model_Data/RI_Auto_Model/250324_RI_Auto_Model_Scored_Data.csv')
scored.shape

(824212, 11)

In [9]:
scored['unique_id']= scored['unique_id'].astype(str)
funded_dev['unique_id']= funded_dev['unique_id'].astype(str)

In [10]:
funded_dev= funded_dev.merge(scored[['unique_id', 'client', 'p1', 'weight']], on= ['unique_id', 'client'], how= 'left')
funded_dev.shape

(209793, 347)

In [11]:
funded_dev[['p1', 'weight']].isnull().sum()

p1        0
weight    0
dtype: int64

In [12]:
funded_dev.rename(columns= {'p1': 'p1_ri'}, inplace= True)

In [13]:
funded_dev[['p1_ri', 'weight']].describe()

,p1_ri,weight
count,209793.000000,209793.000000
mean,0.646481,1.669757
std,0.151359,0.587157
min,0.067892,1.060974
25%,0.542640,1.307603
50%,0.672004,1.488086
75%,0.764758,1.842841
max,0.942530,14.729193


In [14]:
from sklearn.model_selection import train_test_split

In [26]:
F1, F2= train_test_split(funded_dev, test_size= 0.3, random_state= 91)
print(F1['bad_flag'].mean(), F2['bad_flag'].mean())
print(sum(F1['bad_flag']*F1['weight'])/F1['weight'].sum(), sum(F2['bad_flag']*F2['weight'])/F2['weight'].sum())

0.015164618160770828 0.015237217579204931
0.02135328698015847 0.02110936204633752


In [27]:
F1.shape, F2.shape

((146855, 347), (62938, 347))

In [28]:
# F1.to_csv('/root/Unified_LN_V2/LN_Auto_V2/Data/2_LN_Auto_V2_Train_Funded_Data_250324.csv', index= False)
# F2.to_csv('/root/Unified_LN_V2/LN_Auto_V2/Data/2_LN_Auto_V2_Test_Funded_Data_250324.csv', index= False)
# Oot[Oot['funded_flag']== 1].to_csv('/root/Unified_LN_V2/LN_Auto_V2/Data/2_LN_Auto_V2_Oot_Funded_Data_250324.csv', index= False)

In [29]:
Oot[Oot['funded_flag']== 1].shape

(37121, 345)

In [30]:
# TTD.to_csv('/root/Unified_LN_V2/LN_Auto_V2/Data/2_LN_Auto_V2_TTD_Data_250324.csv', index= False)